In [2]:
import os
import sys
import pandas as pd
pd.set_option('display.max_rows', None)

current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

from swp.utils.setup import seed_everything, set_device
from swp.datasets.phonemes import get_phoneme_to_id

seed_everything()
device = set_device()
phoneme_to_id = get_phoneme_to_id()


Using MPS device


In [ ]:
from ast import literal_eval

complete_train = pd.read_csv("../stimuli/dataframe/complete_train.csv")
raw_pseudo = pd.read_csv("../stimuli/handmade/test_data/raw_pseudo.csv")
print(len(raw_pseudo))

# filter out all words that are in the train set
raw_pseudo = raw_pseudo[~raw_pseudo["Word"].isin(complete_train["Word"])]
print(len(raw_pseudo))





11000
10239


### Generating Evaluation Dataset

In [ ]:
from ast import literal_eval

complete_train = pd.read_csv("../stimuli/dataframe/complete_train.csv")
complete_train["Lexicality"] = "real"
complete_train["Phonemes"] = complete_train["Phonemes"].apply(literal_eval)
complete_train["Length"] = complete_train["Phonemes"].apply(len)

# divide by length
short = complete_train.query("2 < Length < 6")
long = complete_train.query("6 < Length < 10")
short["Size"] = "short"
long["Size"] = "long"

# divide by frequency
lh = long[long["Zipf Frequency"] >= 4]
ll = long[long["Zipf Frequency"] <= 3.5]
sh = short[short["Zipf Frequency"] >= 4]
sl = short[short["Zipf Frequency"] <= 3.5]

# save files if necessary
# lh.to_csv("./long_high.csv", index=False)
# ll.to_csv("./long_low.csv", index=False)
# sh.to_csv("./short_high.csv", index=False)
# sl.to_csv("./short_low.csv", index=False)

In [2]:
rlch = pd.read_csv("../stimuli/handmade/test_data/rlch.csv", index_col=0).sample(100)
rlcl = pd.read_csv("../stimuli/handmade/test_data/rlcl.csv", index_col=0).sample(100)
rlsh = pd.read_csv("../stimuli/handmade/test_data/rlsh.csv", index_col=0).sample(100)
rlsl = pd.read_csv("../stimuli/handmade/test_data/rlsl.csv", index_col=0).sample(100)
rsch = pd.read_csv("../stimuli/handmade/test_data/rsch.csv", index_col=0).sample(100)
rscl = pd.read_csv("../stimuli/handmade/test_data/rscl.csv", index_col=0).sample(100)
rssh = pd.read_csv("../stimuli/handmade/test_data/rssh.csv", index_col=0).sample(100)
rssl = pd.read_csv("../stimuli/handmade/test_data/rssl.csv", index_col=0).sample(100)
print(len(rlch), len(rlcl), len(rlsh), len(rlsl), len(rsch), len(rscl), len(rssh), len(rssl))

rlch["Condition"] = "RLCH"
rlch["Morphology"] = "complex"
rlcl["Condition"] = "RLCL"
rlcl["Morphology"] = "complex"
rlsh["Condition"] = "RLSH"
rlsh["Morphology"] = "simple"
rlsl["Condition"] = "RLSL"
rlsl["Morphology"] = "simple"
rsch["Condition"] = "RSCH"
rsch["Morphology"] = "complex"
rscl["Condition"] = "RSCL"
rscl["Morphology"] = "complex"
rssh["Condition"] = "RSSH"
rssh["Morphology"] = "simple"
rssl["Condition"] = "RSSL"
rssl["Morphology"] = "simple"

plc = pd.read_csv("../stimuli/handmade/test_data/plc.csv", index_col=0).sample(100)
pls = pd.read_csv("../stimuli/handmade/test_data/pls.csv", index_col=0).sample(100)
psc = pd.read_csv("../stimuli/handmade/test_data/psc.csv", index_col=0).sample(100)
pss = pd.read_csv("../stimuli/handmade/test_data/pss.csv", index_col=0).sample(100)
print(len(plc), len(pls), len(psc), len(pss))

test_equalized = pd.concat([rlch, rlcl, rlsh, rlsl, rsch, rscl, rssh, rssl, plc, pls, psc, pss], ignore_index=True)
order = [
    "Word", "Condition", "Lexicality", "Size", "Morphology",
    "Frequency", "Length", "Zipf Frequency", "Phonemes",
    "No Stress", "Part of Speech"
]
test_equalized = test_equalized[order]
print(test_equalized["Condition"].value_counts())

test_equalized.to_csv("../stimuli/handmade/test_equalized.csv")

100 100 100 100 100 100 100 100
100 100 100 100
Condition
RLCH    100
RLCL    100
RLSH    100
RLSL    100
RSCH    100
RSCL    100
RSSH    100
RSSL    100
PLC     100
PLS     100
PSC     100
PSS     100
Name: count, dtype: int64


In [7]:
plc = pd.read_csv("../stimuli/handmade/test_data/plc.csv", index_col=0)
pls = pd.read_csv("../stimuli/handmade/test_data/pls.csv", index_col=0)
psc = pd.read_csv("../stimuli/handmade/test_data/psc.csv", index_col=0)
pss = pd.read_csv("../stimuli/handmade/test_data/pss.csv", index_col=0)

pseudo_complete = pd.concat([plc, pls, psc, pss], ignore_index=True)
print(pseudo_complete[["Length", "Morphology"]].sort_values("Length").value_counts())

Length  Morphology
4       simple        161
6       simple        155
5       complex       131
7       simple        123
5       simple        119
4       complex       116
6       complex       101
3       simple         89
7       complex        72
8       simple         66
3       complex        59
8       complex        55
9       complex        20
        simple         15
Name: count, dtype: int64
